In [2]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
import time

In [2]:
class_name = ['Thumbs Up','Thumbs Down','Peace','Super','You','Call Me','Rock','Love You','High Five','None']

In [3]:
# data_path = r"D:\cv dataset\handgesture"
# for name in class_name:
#     dir = os.path.join(data_path,name)
#     os.makedirs(dir)

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawings = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable= False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [11]:
cap = cv2.VideoCapture(0)
num = 1
j = 0 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence =0.5,) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        # Get the frame and results from Mediapipe
        image, results = mediapipe_detection(frame, holistic)

        if results.left_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            try:
                if cv2.waitKey(1) & 0xFF == ord(' ') and num <=200:
                    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
                    data_path = r"D:\cv dataset\handgesture"
                    npy_path = os.path.join(data_path,class_name[j],str(num))
            
#                     np.save(npy_path, lh)
                    num+=1
                elif num > 200:
                    j+=1
                    num = 0
            except:
                break
                
           
        frame = cv2.flip(image, 1)
        cv2.putText(frame,f'collecting {class_name[j]} frame:{str(num)}',(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        cv2.imshow('sample', frame)

        # Breaking Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()

In [15]:
data_path = r"D:\cv dataset\handgesture"
data, labels = [], []
for label in class_name:
    for num in np.array(os.listdir(os.path.join(data_path, label))):
        res = np.load(os.path.join(data_path, label, num))
        data.append(res)
        labels.append(label)

In [16]:
len(data), len(labels)

(2005, 2005)

In [17]:
x = np.array(data)
x.shape

(2005, 63)

In [18]:
y = np.array(labels)
y.shape

(2005,)

In [19]:
y[:10]

array(['Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up',
       'Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up'],
      dtype='<U11')

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)

In [21]:
y[:10]

array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [22]:
label_names = []
for i in range(10):
    name = le.inverse_transform([i])
    label_names.append(name[0])

In [23]:
label_names

['Call Me',
 'High Five',
 'Love You',
 'None',
 'Peace',
 'Rock',
 'Super',
 'Thumbs Down',
 'Thumbs Up',
 'You']

In [24]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=0.2, random_state=42)

In [25]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1604, 63), (401, 63), (1604,), (401,))

In [26]:
x_train[0].shape

(63,)

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=2, activation="relu", input_shape=(63, 1)),
    tf.keras.layers.MaxPool1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 62, 64)            192       
                                                                 
 max_pooling1d (MaxPooling1  (None, 31, 64)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 1984)              0         
                                                                 
 dense (Dense)               (None, 128)               254080    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                        

In [29]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])

In [31]:
model.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=1000)

Epoch 1/1000


51/51 [==============================] - 4s 24ms/step - loss: 2.2505 - acc: 0.1727 - val_loss: 2.1331 - val_acc: 0.3092
Epoch 2/1000
51/51 [==============================] - 1s 12ms/step - loss: 1.8204 - acc: 0.4202 - val_loss: 1.5176 - val_acc: 0.3865
Epoch 3/1000
51/51 [==============================] - 1s 15ms/step - loss: 1.1881 - acc: 0.5998 - val_loss: 0.9922 - val_acc: 0.6683
Epoch 4/1000
51/51 [==============================] - 1s 12ms/step - loss: 0.8523 - acc: 0.7026 - val_loss: 0.7688 - val_acc: 0.7207
Epoch 5/1000
51/51 [==============================] - 1s 10ms/step - loss: 0.6569 - acc: 0.7830 - val_loss: 0.5850 - val_acc: 0.8579
Epoch 6/1000
51/51 [==============================] - 1s 14ms/step - loss: 0.5175 - acc: 0.8348 - val_loss: 0.4636 - val_acc: 0.9002
Epoch 7/1000
51/51 [==============================] - 1s 24ms/step - loss: 0.4219 - acc: 0.8759 - val_loss: 0.3891 - val_acc: 0.9426
Epoch 8/1000
51/51 [==============================] - 1s 10ms/step 

Epoch 59/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0268 - acc: 0.9906 - val_loss: 0.0344 - val_acc: 0.9875
Epoch 60/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0194 - acc: 0.9950 - val_loss: 0.0252 - val_acc: 0.9925
Epoch 61/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0197 - acc: 0.9944 - val_loss: 0.0461 - val_acc: 0.9825
Epoch 62/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0108 - acc: 0.9994 - val_loss: 0.0231 - val_acc: 0.9925
Epoch 63/1000
51/51 [==============================] - 0s 7ms/step - loss: 0.0108 - acc: 0.9963 - val_loss: 0.0311 - val_acc: 0.9875
Epoch 64/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0099 - acc: 0.9988 - val_loss: 0.0202 - val_acc: 0.9950
Epoch 65/1000
51/51 [==============================] - 0s 8ms/step - loss: 0.0137 - acc: 0.9950 - val_loss: 0.0237 - val_acc: 0.9900
Epoch 66/1000
51/51 [==============================] - 0s 8ms/step - 

KeyboardInterrupt: 

In [32]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 0s 4ms/step - loss: 0.0118 - acc: 0.9950


[0.011783341877162457, 0.9950124621391296]

In [34]:
# model.save("C:\\Users\\vicky\\desktop\\sample_project_1\\saved_models\\left_hand_gesturev3.h5")

C:\Users\vicky\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
label_names = ['Call Me', 'High Five', 'Love You', 'None', 'Peace', 'Rock', 'Super', 'Thumbs Down', 'Thumbs Up', 'You']

In [6]:
left_hand_model = tf.keras.models.load_model("C:\\Users\\vicky\\desktop\\sample_project_1\\saved_models\\left_hand_gesturev3.h5")
right_hand_model = tf.keras.models.load_model("C:\\Users\\vicky\\desktop\\sample_project_1\\saved_models\\handgesture_rightv2.h5")

In [7]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence =0.5,) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        # Get the frame and results from Mediapipe
        image, results = mediapipe_detection(frame, holistic)

        if results.left_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                      mp_drawings.DrawingSpec(color=(200,22,76), thickness=2, circle_radius=2), 
                                      mp_drawings.DrawingSpec(color=(100,44,100), thickness=2, circle_radius=1))
            left_arr = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
            left_arr_reshaped = np.expand_dims(left_arr, 0)
            predicted_left = left_hand_model.predict(left_arr_reshaped)
            left_gesture = label_names[np.argmax(predicted_left)]
            print(f"LEFT: {left_gesture}")
            
        if results.right_hand_landmarks:
            
            mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                       mp_drawings.DrawingSpec(color=(166,117,66), thickness=2, circle_radius=2), 
                                       mp_drawings.DrawingSpec(color=(188,66,77), thickness=2, circle_radius=1))
            right_arr = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
            right_arr_reshaped = np.expand_dims(right_arr, 0)
            predicted_right = right_hand_model.predict(right_arr_reshaped)
            right_gesture = label_names[np.argmax(predicted_right)]
            print(f"RIGHT: {right_gesture}")
            
                
           
        image = cv2.flip(image, 1)
        if results.left_hand_landmarks:
            cv2.putText(image,left_gesture,(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(188,66,77),3,cv2.LINE_AA)
        if results.right_hand_landmarks:
            cv2.putText(image,right_gesture,(400,30),cv2.FONT_HERSHEY_COMPLEX,1,(188,66,77),3,cv2.LINE_AA)
        if not results.left_hand_landmarks and results.right_hand_landmarks:
            cv2.putText(image," ",(15,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
        cv2.imshow('sample', image)

        # Breaking Gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\vicky\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 [==============================] - 1s 807ms/step
LEFT: None
1/1 [==============================] - 0s 151ms/step
RIGHT: None
1/1 [==============================] - 0s 32ms/step
RIGHT: None
1/1 [==============================] - 0s 25ms/step
RIGHT: None
1/1 [==============================] - 0s 36ms/step
RIGHT: None
1/1 [==============================] - 0s 38ms/step
RIGHT: Super
1/1 [==============================] - 0s 32ms/step
RIGHT: Super
1/1 [==============================] - 0s 32ms/step
RIGHT: Super
1/1 [==============================] - 0s 20ms/step
RIGHT: Super
1/1 [==============================] - 0s 25ms/step
RIGHT: Super
1/1 [==============================] - 0s 25ms/step
RIGHT: Super
1/1 [==============================] - 0s 24ms/step
RIGHT: Super
1/1 [==============================] - 0s 30ms/step
RIGHT: Super
1/1 [==============================] - 0s 31ms/step
RIGHT: Super
1/1 [==============================] - 0s 51ms/step
RIGHT: Super
1/1 [========================

1/1 [==============================] - 0s 24ms/step
LEFT: None
1/1 [==============================] - 0s 45ms/step
RIGHT: Super
1/1 [==============================] - 0s 31ms/step
RIGHT: Peace
1/1 [==============================] - 0s 30ms/step
RIGHT: Peace
1/1 [==============================] - 0s 28ms/step
RIGHT: High Five
1/1 [==============================] - 0s 26ms/step
RIGHT: High Five
1/1 [==============================] - 0s 28ms/step
RIGHT: High Five
1/1 [==============================] - 0s 31ms/step
RIGHT: None
1/1 [==============================] - 0s 27ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 23ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 46ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 29ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 32ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 27ms/step
RIGHT

1/1 [==============================] - 0s 40ms/step
RIGHT: High Five
1/1 [==============================] - 0s 32ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 25ms/step
RIGHT: High Five
1/1 [==============================] - 0s 36ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 36ms/step
RIGHT: High Five
1/1 [==============================] - 0s 36ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 21ms/step
RIGHT: None
1/1 [==============================] - 0s 30ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 34ms/step
RIGHT: None
1/1 [==============================] - 0s 30ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 32ms/step
RIGHT: None
1/1 [==============================] - 0s 29ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 29ms/step
RIGHT: None
1/1 [==============================] - 0s 31ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 28ms/step
RIGHT: No

1/1 [==============================] - 0s 39ms/step
LEFT: Call Me
1/1 [==============================] - 0s 21ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 28ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 27ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 37ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 33ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 32ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 25ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 39ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 23ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 23ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 32ms/step
RIGHT: Thumbs Down
1/1 [==============================] - 0s 32ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 28ms/step
RIGHT: Thumbs Down
1/1 [========================

1/1 [==============================] - 0s 38ms/step
LEFT: None
1/1 [==============================] - 0s 42ms/step
RIGHT: None
1/1 [==============================] - 0s 24ms/step
LEFT: None
1/1 [==============================] - 0s 30ms/step
RIGHT: None
1/1 [==============================] - 0s 40ms/step
LEFT: High Five
1/1 [==============================] - 0s 28ms/step
RIGHT: High Five
1/1 [==============================] - 0s 29ms/step
LEFT: None
1/1 [==============================] - 0s 29ms/step
RIGHT: None
1/1 [==============================] - 0s 24ms/step
LEFT: None
1/1 [==============================] - 0s 32ms/step
RIGHT: None
1/1 [==============================] - 0s 31ms/step
LEFT: None
1/1 [==============================] - 0s 29ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 25ms/step
LEFT: None
1/1 [==============================] - 0s 31ms/step
LEFT: None
1/1 [==============================] - 0s 34ms/step
LEFT: None
1/1 [==============================

1/1 [==============================] - 0s 22ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 20ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 28ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 24ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 22ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 25ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 26ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 21ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 20ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 20ms/step
LEFT: None
1/1 [==============================] - 0s 21ms/step
LEFT: None
1/1 [==============================] - 0s 24ms/step
LEFT: None
1/1 [==============================] - 0s 27ms/step
LEFT: None
1/1 [==============================] - 0s 22ms/step
L

1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 23ms/step
LEFT: None
1/1 [==============================] - 0s 23ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 23ms/step
LEFT: None
1/1 [==============================] - 0s 19ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 25ms/step
LEFT: None
1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 23ms/step
LEFT: None
1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 20ms/step
LEFT: None
1/1 [==============================] - 0s 21ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 19ms/step
LEFT: None
1/1 [==============================] - 0s 21ms/step
RIGHT: Thumbs Up
1/1 [==============================] - 0s 20ms/step
LEFT: None
1/1 [==============================] - 0s 22ms/step
RIGHT: Thumbs Up
1/1 [==

1/1 [==============================] - 0s 25ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 21ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 20ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 23ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 24ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 22ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 25ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 22ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 20ms/step
LEFT: Thumbs Down
1/1 [==============================] - 0s 24ms/step
LEFT: Peace
1/1 [==============================] - 0s 20ms/step
LEFT: Rock
1/1 [==============================] - 0s 22ms/step
LEFT: None
1/1 [==============================] - 0s 22ms/step
LEFT: None
1/1 [==============================] - 0s 21ms/step
LEFT: Thumbs Up
1/1 [==============================] - 0s 21ms/st

In [7]:
cap.release()
cv2.destroyAllWindows()

In [48]:
left_arr

array([ 1.04487360e+00,  1.06351304e+00,  2.80475447e-08,  9.53214943e-01,
        1.02653134e+00, -4.13634703e-02,  8.76197219e-01,  9.40644026e-01,
       -6.38309270e-02,  8.19850802e-01,  8.57002378e-01, -8.15095976e-02,
        7.72206664e-01,  7.95441151e-01, -9.79815349e-02,  9.27148998e-01,
        8.04357827e-01, -4.54015546e-02,  8.81987333e-01,  6.83759570e-01,
       -7.56919309e-02,  8.56817722e-01,  6.16020560e-01, -9.93746668e-02,
        8.36728752e-01,  5.58482885e-01, -1.15266554e-01,  9.85207021e-01,
        7.72324204e-01, -4.19895984e-02,  9.66569185e-01,  6.33554399e-01,
       -6.55072331e-02,  9.50432539e-01,  5.53283274e-01, -8.39131400e-02,
        9.36568379e-01,  4.88407254e-01, -9.81618538e-02,  1.03419316e+00,
        7.68848777e-01, -4.27572392e-02,  1.02586877e+00,  6.43190444e-01,
       -6.68501630e-02,  1.01210129e+00,  5.69403887e-01, -7.83418491e-02,
        9.97948349e-01,  5.11881113e-01, -8.56574923e-02,  1.07390046e+00,
        7.82405615e-01, -

In [52]:
x_test[0]

array([ 6.08930469e-01,  7.06825137e-01, -6.11382063e-07,  5.21164894e-01,
        7.26022482e-01, -1.96498614e-02,  4.50665742e-01,  7.18310714e-01,
       -3.80489007e-02,  4.07115400e-01,  6.98190629e-01, -5.56910522e-02,
        4.07400638e-01,  6.67146802e-01, -6.74697757e-02,  4.43074346e-01,
        5.18602312e-01, -2.03063004e-02,  3.92431170e-01,  5.91622829e-01,
       -4.92603406e-02,  4.23693448e-01,  6.41198933e-01, -6.29584715e-02,
        4.46384013e-01,  6.38049781e-01, -7.07482174e-02,  4.80251998e-01,
        5.02970219e-01, -2.72896588e-02,  4.28977609e-01,  6.02255464e-01,
       -5.52987829e-02,  4.60656285e-01,  6.52786374e-01, -5.67631498e-02,
        4.81318384e-01,  6.45734966e-01, -5.39139174e-02,  5.20486593e-01,
        5.13019085e-01, -3.85574140e-02,  4.71754164e-01,  6.19819582e-01,
       -6.52442798e-02,  4.99344707e-01,  6.66307211e-01, -5.29251322e-02,
        5.18352389e-01,  6.57706082e-01, -4.05499898e-02,  5.58833420e-01,
        5.42473257e-01, -

In [53]:
left_arr_reshaped = left_arr.reshape(1, left_arr.shape[0], 1)
model.predict(left_arr_reshaped)

1/1 [==============================] - 0s 73ms/step


array([[6.6124558e-13, 4.7000164e-01, 6.4897317e-06, 7.3568624e-01,
        1.9521780e-02, 1.4103352e-03, 4.9747800e-04, 1.4675743e-09,
        7.5232261e-04, 7.5088000e-14]], dtype=float32)